## Phase One Project Submission

* Student name: Lydia Cuffman, Ramses Nestor Reis, Michael Romanski
* Student pace: full time
* Scheduled project review date/time: July 14, 2023
* Instructor name: Daniel Burdeno

### Overview

Our client is looking to enter the aviation industry and has asked us to determine how to maximize safety in this endeavor. Using data provided by the National Transportation Safety Board (NTSB) we have determined the following:

*Our client should invest in a Boeing aircraft that has at least two engines. 

*Our client should fly on-demand, unscheduled operations under Federal Aviation Regulation Part 135. 

*Our client should pursue regional business in the United States but outside of the Northeast.


### Business Understanding

Our client's top concern is safety. They want to know what type of aircraft is safest and how to ensure it flies safely. This benefits their bottom line too, because a focus on safety can burnish their brand in the eyes of customers. Safe flights also protect their capital investment.

### Data Understanding

This data represents aviation accidents and incidents documented by the NTSB from 1968 to present. It provides details about the make and model of the aircraft involved as well as the type and number of engines. The data include numbers of injuries, categorized by severity. Entries also detail the location of the accident, the rules the plane was flying under, the date of the incident, and whether the pilot could fly visually (good weather) or with instruments (poor weather).

The biggest limitation of this dataset is that it only includes flights that had a problem. Theoretically, a perfect plane would never crash and thus would not appear in this dataset. While the data did not allow us to determine the ratio of uneventful flights to crashes, we were able to find valuable information about the severity of crashes. Our analysis shows the likelihood of injury if a particular plane does crash. Realistically, accidents do happen and there are no perfect aircraft. We can tell our client which planes are the safest when things go wrong.

### Data Analysis

In [1]:
#Import Packages
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# Loading in Aviation_data.csv
df = pd.read_csv('data/Aviation_Data.csv')

/Users/michaelromanski/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (6,7,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# Creating a new DataFrame to keep original intact
aviation_df = df.copy()

We quickly eliminated columns that were not germane to our analysis. For example, our client is not seeking to purchase an already existing commercial airline, so "Air.carrier" was not useful. "Broad.phase.of.flight" would probably yield good information about the relative safety of various parts of a flight, but we wouldn't be able to recommend to our client that they avoid takeoffs and landings altogether, should we discover they were the most dangerous phases. Some categories like "Registration.Number" and "Investigation.Type" were more connected to internal NTSB processes.

In [4]:
# Selecting relevant columns for analysis
aviation_df = aviation_df.drop(['Latitude', 'Longitude', 'Airport.Code', 'Airport.Name', 'Aircraft.Category', 'Publication.Date', 'Air.carrier', 'Broad.phase.of.flight', 'Event.Id', 'Accident.Number', 'Investigation.Type', 'Registration.Number', 'Schedule'], axis=1)
aviation_df                               

,Event.Date,Location,Country,Injury.Severity,Aircraft.damage,Make,Model,Amateur.Built,Number.of.Engines,Engine.Type,FAR.Description,Purpose.of.flight,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Report.Status
0,1948-10-24,"MOOSE CREEK, ID",United States,Fatal(2),Destroyed,Stinson,108-3,No,1.0,Reciprocating,NaN,Personal,2.0,0.0,0.0,0.0,UNK,Probable Cause
1,1962-07-19,"BRIDGEPORT, CA",United States,Fatal(4),Destroyed,Piper,PA24-180,No,1.0,Reciprocating,NaN,Personal,4.0,0.0,0.0,0.0,UNK,Probable Cause
2,1974-08-30,"Saltville, VA",United States,Fatal(3),Destroyed,Cessna,172M,No,1.0,Reciprocating,NaN,Personal,3.0,NaN,NaN,NaN,IMC,Probable Cause
3,1977-06-19,"EUREKA, CA",United States,Fatal(2),Destroyed,Rockwell,112,No,1.0,Reciprocating,NaN,Personal,2.0,0.0,0.0,0.0,IMC,Probable Cause
4,1979-08-02,"Canton, OH",United States,Fatal(1),Destroyed,Cessna,501,No,NaN,NaN,NaN,Personal,1.0,2.0,NaN,0.0,VMC,Probable Cause
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90343,2022-12-26,"Annapolis, MD",United States,Minor,NaN,PIPER,PA-28-151,No,NaN,NaN,091,Personal,0.0,1.0,0.0,0.0,NaN,NaN
90344,2022-12-26,"Hampton, NH",United States,NaN,NaN,BELLANCA,7ECA,No,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN
90345,2022-12-26,"Payson, AZ",United States,Non-Fatal,Substantial,AMERICAN CHAMPION AIRCRAFT,8GCBC,No,1.0,NaN,091,Personal,0.0,0.0,0.0,1.0,VMC,NaN
90346,2022-12-26,"Morgan, UT",United States,NaN,NaN,CESSNA,210N,No,NaN,NaN,091,Personal,0.0,0.0,0.0,0.0,NaN,NaN


Then we began cleaning the data.

In [5]:
# Replacing dot to space in column names
aviation_df = aviation_df.rename(columns=lambda x: x.replace('.', ' '))

In [6]:
# Capitalizing all values in Make
aviation_df['Make'] = aviation_df['Make'].str.capitalize()

In [7]:
# Capitalizing all columns
aviation_df = aviation_df.rename(columns=str.capitalize)

We made a new data frame of U.S. states and their region, and prepped the aviation_df to join it.

In [8]:
# Changing location to just the state abbreviation
aviation_df['Location'] = aviation_df['Location'].str[-2:]

In [9]:
# Reading in data that pairs each state abbreviation with its census-designated region
states_df = pd.read_csv('data/Census_regions.csv')

### Data from here: https://github.com/cphalpert/census-regions/blob/master/us%20census%20bureau%20regions%20and%20divisions.csv

In [10]:
states_df = states_df.drop(['State','Division'], axis=1)

In [11]:
states_df.set_index('State Code', inplace=True)

In [12]:
aviation_df = aviation_df.rename(columns={'Location':'State Code'})

In [13]:
aviation_df = aviation_df.join(states_df, on='State Code', how='left')

The FAR descriptions were messy and inconsistent. We needed to do a lot of renaming to properly categorize them.

In [14]:
aviation_df.rename(columns={'Far description': 'FAR description'}, inplace=True)

In [15]:
aviation_df.groupby('FAR description').sum()

,Number of engines,Total fatal injuries,Total serious injuries,Total minor injuries,Total uninjured
FAR description,,,,,
091,18655.0,5450.0,4741.0,3197.0,19713.0
091K,24.0,0.0,4.0,1.0,42.0
103,2.0,0.0,0.0,2.0,0.0
107,8.0,0.0,0.0,1.0,6.0
121,1221.0,68.0,777.0,250.0,64254.0
125,10.0,4.0,0.0,1.0,62.0
129,147.0,55.0,211.0,76.0,12828.0
133,110.0,34.0,25.0,31.0,61.0
135,995.0,395.0,285.0,193.0,2455.0


In [16]:
aviation_df["FAR description"]= aviation_df["FAR description"].replace("Part 91: General Aviation", "091")
aviation_df["FAR description"]= aviation_df["FAR description"].replace("Part 91F: Special Flt Ops.", "091")
aviation_df["FAR description"]= aviation_df["FAR description"].replace("091K", "091")
aviation_df["FAR description"]= aviation_df["FAR description"].replace("NUSC", "Non-U.S., Commercial")
aviation_df["FAR description"]= aviation_df["FAR description"].replace("NUSN", "Non-U.S., Non-Commercial")
aviation_df["FAR description"]= aviation_df["FAR description"].replace("Part 121: Air Carrier", "121")
aviation_df["FAR description"]= aviation_df["FAR description"].replace("Part 125: 20+ Pax,6000+ lbs", "125")
aviation_df["FAR description"]= aviation_df["FAR description"].replace("Part 129: Foreign", "129")
aviation_df["FAR description"]= aviation_df["FAR description"].replace("Part 133: Rotorcraft Ext. Load", "133")
aviation_df["FAR description"]= aviation_df["FAR description"].replace("Part 135: Air Taxi & Commuter", "135")
aviation_df["FAR description"]= aviation_df["FAR description"].replace("Part 137: Agricultural", "137")
aviation_df["FAR description"]= aviation_df["FAR description"].replace("PUBU", "Public Use")
aviation_df["FAR description"]= aviation_df["FAR description"].replace("Public Aircraft", "Public Use")
aviation_df["FAR description"]= aviation_df["FAR description"].replace("UNK", "Unknown")

We didn't find any entries with nulls listed for all injury categories, so we decided that null values likely represented zeroes. Since every entry had at least some data for injury counts or uninjured passengers, it seemed likely that missing values were zero.

In [17]:
# Filling all Total Injury-related columns Null data with 0
aviation_df.fillna(value={'Total fatal injuries': 0, 'Total serious injuries': 0, 'Total minor injuries': 0, 'Total uninjured': 0}, inplace=True)

In [18]:
#Drop Amateur Built projects because we will not be recommending amateur products to our client. 
aviation_df.drop(aviation_df[aviation_df['Amateur built'] == 'Yes'].index, inplace = True)

In [19]:
#No longer need column since all remaining entries are professionally built
aviation_df = aviation_df.drop(['Amateur built'], axis = 1)

Our client values safety above all, and we cannot really determine the safety of unique aircraft. It is unlikely that our client would have the opportunity to purchase a rare or limited-production plane anyway, so we dropped all "Make" values that were less than 1% of the total dataset. Such planes are too unusual to make a good assessment of their safety. We also dropped entries that did not list "Make" because we cannot recommend a plane if we don't know what type it is.

In [20]:
# Drop the values in 'Make' that constitute less than 1% of the total data in 'Make' column
aviation_df = aviation_df[~aviation_df['Make'].isin(aviation_df['Make'].value_counts()[aviation_df['Make'].value_counts() < 84].index)]

In [21]:
# Dropping all rows containing null values from Make, as without a Plane make, the other information is invalid
df.dropna(subset=['Make'], inplace=True)

In [22]:
#Change “Number of engines” nulls to 1, as that is the mode and has a minimal impact on the data distribution
aviation_df['Number of engines'].fillna(1, inplace=True)

In [23]:
aviation_df.dropna(subset = ['Year'], inplace=True)

KeyError: ['Year']

In [ ]:
# Filling in remaining null values in aviation_df with Unknown
aviation_df.fillna('Unknown', inplace=True)

We wanted to determine the chance of injury if a plane was in a crash, so we estimated total passengers per flight by adding all the injured passenger counts to the uninjured count. Each category ("fatal," "serious," "minor," and "uninjured") seemed exclusive, so adding them together gives a good indication of the number of passengers on board.

In [ ]:
aviation_df['Total passengers'] = aviation_df['Total fatal injuries'] + aviation_df['Total serious injuries'] + aviation_df['Total minor injuries'] + aviation_df['Total uninjured']

In [ ]:
aviation_df = aviation_df[aviation_df['Total passengers'] != 0]

With our new metrics for injuries, we looked at percentage chance of injury by number of engines.

In [ ]:
grouped_df = aviation_df.groupby("Number of engines").agg({"Total fatal injuries": "sum", "Total serious injuries": "sum", "Total minor injuries": "sum", "Total uninjured": "sum", "Total passengers": "sum"})

In [ ]:
grouped_df["Percentage chance of fatal injury"] = (grouped_df["Total fatal injuries"] / grouped_df["Total passengers"]) * 100
grouped_df["Percentage chance of serious injuries"] = (grouped_df["Total serious injuries"] / grouped_df["Total passengers"]) * 100
grouped_df["Percentage chance of minor injuries"] = (grouped_df["Total minor injuries"] / grouped_df["Total passengers"]) * 100
grouped_df["Percentage chance of being uninjured"] = (grouped_df["Total uninjured"] / grouped_df["Total passengers"]) * 100

In [ ]:
grouped_df['Overall chance of injury'] = grouped_df["Percentage chance of fatal injury"] + grouped_df["Percentage chance of serious injuries"] + grouped_df["Percentage chance of minor injuries"] 

In [ ]:
grouped_df.head()

Yikes! Passengers on one-engine planes have a way higher chance of fatality and any other kind of injury. The same is true of zero-engine aircraft, though it was unlikely we would recommend our client invest in hot air balloons. We opted to exclude zero and one engine aircraft from our data set, because as a category they were far more dangerous.

In [ ]:
aviation_df = aviation_df[(aviation_df['Number of engines'] != 0) & (aviation_df['Number of engines'] != 1)]

With our curated list of planes, we then tried to figure out which make was the safest.

In [ ]:
# Calculate the total number of accidents for each make 
accidents_count = aviation_df.groupby(["Make"]).size().reset_index(name="total_accidents")
accidents_count

In [ ]:
#A few of these makes are listed twice. Let's clean that up.
aviation_df["Make"]= aviation_df["Make"].replace("Airbus industrie", "Airbus")
aviation_df["Make"]= aviation_df["Make"].replace("Dehavilland", "De havilland")

In [ ]:
# Calculate the total number of injuries for each make 
injuries_count = aviation_df.groupby(["Make"])[["Total fatal injuries", "Total serious injuries", "Total minor injuries", 'Total uninjured']].sum().reset_index()
injuries_count

In [ ]:
# Calculate the percentage of accidents resulting in injuries for each make 
injury_percentage = pd.merge(injuries_count, accidents_count, on=["Make"])
total_people = injuries_count['Total fatal injuries'] + injuries_count['Total serious injuries'] + injuries_count['Total minor injuries'] + injuries_count['Total uninjured']
injury_percentage["Fatal percentage"] = (injury_percentage["Total fatal injuries"] / total_people) * 100
injury_percentage["Serious percentage"] = (injury_percentage["Total serious injuries"] / total_people) * 100
injury_percentage["Minor percentage"] = (injury_percentage["Total minor injuries"] / total_people) * 100
injury_percentage["Uninjured percentage"] = (injury_percentage["Total uninjured"] / total_people) * 100
injury_percentage

In [ ]:
injury_percentage["overall_injury_percentage"] = (injury_percentage["Total fatal injuries"] + injury_percentage["Total serious injuries"] + injury_percentage["Total minor injuries"]) / total_people * 100

In [ ]:
injury_percentage

Some planes had an injury rate of zero, but the make had only ever recorded one accident. We excluded models that had not reported many incidents because we did not have sufficient data to determine if they were consistently safe.

In [ ]:
# Filter the injury_percentage DataFrame to include only aircrafts with a significant number of accidents
significant_accidents = injury_percentage[injury_percentage["total_accidents"] >= 100]

# Sort the significant_accidents DataFrame by the lowest overall percentage
safest_overall_aircrafts = significant_accidents.sort_values("overall_injury_percentage")
safest_overall_aircrafts

It looks like if you're going to be in a plane crash, you want to do it in a Boeing! They have recorded a high number of injuries, but that's because there are so many Boeings flying each day. Per incident, the injury rate is very low, as is the fatality rate. Mcdonnell Douglas and Airbus also posted very low overall injury percentages.

In [ ]:
safest_overall_aircrafts.plot.bar(x='Make', y='overall_injury_percentage')
plt.xlabel('Make')
plt.ylabel('Overall Injury Percentage')
plt.title('Correlation between Make and Overall Injury Percentage')
plt.show()

We then took our top ten safest makes and looked back at number of engines.

In [ ]:
aviation_df = aviation_df[(aviation_df['Make'] == 'Boeing') | (aviation_df['Make'] == 'Mcdonnell douglas') 
                          | (aviation_df['Make'] == 'Airbus') | (aviation_df['Make'] == 'Aerospatiale') | 
                          (aviation_df['Make'] == 'Embraer') | (aviation_df['Make'] == 'Douglas') |
                          (aviation_df['Make'] == 'Swearingen') | (aviation_df['Make'] == 'De havilland') |
                          (aviation_df['Make'] == 'Learjet') | (aviation_df['Make'] == 'Beech')]
                          

In [ ]:
top_10_df = aviation_df.groupby(["Number of engines", "Make"]).agg({"Total fatal injuries": "sum", "Total serious injuries": "sum", "Total minor injuries": "sum", "Total uninjured": "sum", "Total passengers": "sum"})

In [ ]:
top_10_df["Percentage chance of fatal injury"] = (top_10_df["Total fatal injuries"] / top_10_df["Total passengers"]) * 100
top_10_df["Percentage chance of serious injuries"] = (top_10_df["Total serious injuries"] / top_10_df["Total passengers"]) * 100
top_10_df["Percentage chance of minor injuries"] = (top_10_df["Total minor injuries"] / top_10_df["Total passengers"]) * 100
top_10_df["Percentage chance of being uninjured"] = (top_10_df["Total uninjured"] / top_10_df["Total passengers"]) * 100

In [ ]:
top_10_df['Overall chance of injury'] = top_10_df["Percentage chance of fatal injury"] + top_10_df["Percentage chance of serious injuries"] + top_10_df["Percentage chance of minor injuries"]

In [ ]:
top_10_df.head(30)

In [ ]:
top_10_df = top_10_df.sort_values(by='Overall chance of injury')

In [ ]:
top_10_df.head(30)

In [ ]:
top_10_df.reset_index()

Our top five safest aircraft are the four-engine Airbus, four-engine De Havilland, two-engine Boeing, two-engine McDonnell Douglas, and the three-engine Boeing. The four-engine Boeings were just outside the top five. We opted to recommend the Boeings and the two-engine McDonnell Douglas because the AirBus and De Havilland had substantially lower incident counts. We wanted to recommend a plane that was repeatedly proven to protect its passengers in the event of an accident.

After determining our pick for safest aircraft, we went back to our regional data to determine how many injuries were associated with a crash in each of our regions.

In [ ]:
aviation_df['Total injuries']=aviation_df['Total fatal injuries']+aviation_df['Total serious injuries']+aviation_df['Total minor injuries']

In [ ]:
region_counts = aviation_df['Region'].value_counts()
region_counts.sort_index(inplace=True)
region_counts

In [ ]:
region_injuries = aviation_df.groupby(['Region'])['Total injuries'].sum()
region_injuries.sort_index(inplace=True)

In [ ]:
region_injury_rate = region_injuries/region_counts
region_injury_rate.sort_index(inplace=True)
region_injury_rate

We determined that a crash in the Northeast had a higher chance of injury. (The "Unknown" category includes incidents outside the U.S. as well as any incident that did not list a location.)

An early exploration of the FAR description data showed that planes flying under Part 91 by far had the most injuries. But Part 91 is also very likely to be one-engine planes, since it is for general aviation, or non-commercial flying. We looked into the data without single-engine planes to see how many accidents were associated with crashes under each FAR protocol.

In [ ]:
far_counts = aviation_df['FAR description'].value_counts()
far_counts.sort_index(inplace=True)
far_counts

In [ ]:
far_injuries = aviation_df.groupby(['FAR description'])['Total injuries'].sum()
far_injuries.sort_index(inplace=True)
far_injuries

In [ ]:
far_injury_rate = far_injuries/far_counts
far_injury_rate.sort_index(inplace=True)
far_injury_rate

The highest injury rates were for Part 129 and for Non-U.S., Commercial, both of which apply to foreign flights. Some of the categories. like military and public use, are not relevant to our client. Of commercial and domestic options, Part 135 had the highest record of safety.

In [ ]:
top_10_df.to_csv('data/top_aircraft.csv')

In [ ]:
aviation_df.to_csv('data/cleaned_aviation.csv')

In [ ]:
grouped_df.to_csv('data/injuries_from_num_engine.csv')

In [ ]:
safest_overall_aircrafts.to_csv('data/injuries_from_make.csv')

In [ ]:
region_injury_rate.to_csv('data/region_injury_rate.csv')

In [ ]:
far_injury_rate.to_csv('data/far_injury_rate.csv')

### Recommendations

We offer our client the following recommendations:

*Invest in Boeing airplanes with two, three, or four engines. A two-engine McDonnell Douglas is also proven safe.

*Focus operations in the United States outside of the Northeast.

*Fly unscheduled, on-demand flights under Federal Aviation Regulation Part 135.

### Next Steps

Next steps for data analysis could include further investigation of the safest models of Boeing aircraft. We could also look more deeply into what might lead to accidents in the Northeast having more severe outcomes. We could use the weather data included in this dataset or seek other data to help us dig into that question.

Next steps for our client would involve researching price and availability of our recommended aircraft. They also might consider opportunities to align this new venture with their commercial interests and their social responsibility values. Many Part 135 operations serve isolated or underserved areas. This offers the company a chance to find untapped markets and possibly to also demonstrate their business' commitment to giving back.
